<a href="https://colab.research.google.com/github/rgrion/mlarrozprevisao/blob/main/Experimentos_Preliminares/Tratamento_Rice_South_America_Nasa_Power.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bloco 1: Leitura dos CSVs já existentes no diretório
import pandas as pd
import glob, os

# Ajuste o padrão se precisar (ex.: "Rice_*.csv")
rice_files = sorted(glob.glob("Rice_South_America_*.csv"))
clima_path = "clima_nasa_power.csv"

if not rice_files:
    raise FileNotFoundError("Não encontrei 'Rice_South_America_*.csv' no diretório atual.")
if not os.path.exists(clima_path):
    raise FileNotFoundError("Não encontrei 'clima_nasa_power.csv' no diretório atual.")

dfs_rice = [pd.read_csv(f) for f in rice_files]
df_rice_raw = pd.concat(dfs_rice, ignore_index=True)
df_clima_raw = pd.read_csv(clima_path)

print(f"{len(rice_files)} arquivos de arroz lidos. df_rice_raw: {df_rice_raw.shape}")
print(f"Arquivo de clima lido. df_clima_raw: {df_clima_raw.shape}")

# Espiadas (opcional)
display(df_rice_raw.head(3))
display(df_clima_raw.head(3))

13 arquivos de arroz lidos. df_rice_raw: (169, 6)
Arquivo de clima lido. df_clima_raw: (240, 4)


,Area,Year,Area harvested (ha),Production (t),Yield (kg/ha),razão de produção/area
0,Argentina,2011,257445.00,1748075.00,6790.10,6790.09
1,Bolivia (Plurinational State of),2011,178273.00,428007.00,2400.90,2400.85
2,Brazil,2011,2752891.00,13476994.00,4895.60,4895.58


,Pais,Ano,Temp_Media_C,Precipitacao_mm
0,Argentina,2004,16.60,37.50
1,Argentina,2005,16.35,25.33
2,Argentina,2006,17.24,31.32


In [ ]:
# Bloco 2: Padronização de colunas e países

df_rice = df_rice_raw.copy()
df_clima = df_clima_raw.copy()

# 2.1 — Padronizar cabeçalhos dos CSVs de arroz
rename_rice = {
    "Area harvested": "Area harvested (ha)",
    "Yield": "Yield (kg/ha)",
    "Production": "Production (t)",
    "Pais": "Area",
    "Ano": "Year",
}
for k, v in rename_rice.items():
    if k in df_rice.columns and v not in df_rice.columns:
        df_rice.rename(columns={k: v}, inplace=True)

expected = ["Area", "Year", "Production (t)", "Area harvested (ha)", "Yield (kg/ha)"]
faltando = [c for c in expected if c not in df_rice.columns]
if faltando:
    raise ValueError(f"Faltam colunas no arroz: {faltando}")

# Remover total agregado "South America", se existir
df_rice = df_rice[df_rice["Area"].str.strip().str.lower() != "south america"].copy()

# Converter tipos e limpar separadores de milhar
def to_numeric_safe(series):
    s = series.astype(str).str.replace(",", "", regex=False).str.strip()
    return pd.to_numeric(s, errors="coerce")

df_rice["Year"] = pd.to_numeric(df_rice["Year"], errors="coerce").astype(int)
for c in ["Production (t)", "Area harvested (ha)", "Yield (kg/ha)"]:
    df_rice[c] = to_numeric_safe(df_rice[c])

# 2.2 — Padronizar cabeçalhos do CLIMA
rename_clima = {
    "País": "Pais", "Country": "Pais",
    "year": "Ano", "Year": "Ano",
    "Temperature_C": "Temp_Media_C",
    "Precip_mm": "Precipitacao_mm",
}
for k, v in rename_clima.items():
    if k in df_clima.columns and v not in df_clima.columns:
        df_clima.rename(columns={k: v}, inplace=True)

req_clima = ["Pais", "Ano", "Temp_Media_C", "Precipitacao_mm"]
faltando_c = [c for c in req_clima if c not in df_clima.columns]
if faltando_c:
    raise ValueError(f"O clima precisa ter {req_clima}. Faltando: {faltando_c}")

df_clima["Ano"] = pd.to_numeric(df_clima["Ano"], errors="coerce").astype(int)
df_clima["Temp_Media_C"] = to_numeric_safe(df_clima["Temp_Media_C"])
df_clima["Precipitacao_mm"] = to_numeric_safe(df_clima["Precipitacao_mm"])

# 2.3 — Normalizar nomes de países (adicione variações se aparecerem no print abaixo)
mapa_paises = {
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Bolivia (Plurinational State of)":   "Bolivia",
    "Venezuela, Bolivarian Republic of":  "Venezuela",
    "Bolivia, Plurinational State of":    "Bolivia",
    # PT/ES -> EN
    "Brasil": "Brazil", "Guiana": "Guyana", "Paraguai": "Paraguay",
    "Peru ": "Peru", "Suriname ": "Suriname"
}

def normaliza_pais(x):
    if pd.isna(x): return x
    x2 = str(x).strip()
    return mapa_paises.get(x2, x2)

df_rice["Area_norm"]  = df_rice["Area"].apply(normaliza_pais)
df_clima["Pais_norm"] = df_clima["Pais"].apply(normaliza_pais)

# Diagnóstico: quem não casa?
nao_batem_rice  = sorted(set(df_rice["Area_norm"])  - set(df_clima["Pais_norm"]))
nao_batem_clima = sorted(set(df_clima["Pais_norm"]) - set(df_rice["Area_norm"]))

print("Países do ARROZ que não aparecem no CLIMA:", nao_batem_rice)
print("Países do CLIMA que não aparecem no ARROZ:", nao_batem_clima)

# Dica: se aparecer algo acima, inclua no mapa_paises e reexecute este bloco.

Países do ARROZ que não aparecem no CLIMA: []
Países do CLIMA que não aparecem no ARROZ: []


In [ ]:
# Bloco 3: Merge final + arredondamento + salvar

# Preparar bases para merge
base_rice  = df_rice.rename(columns={"Area_norm": "Pais_norm"}).copy()
base_clima = df_clima[["Pais_norm", "Ano", "Temp_Media_C", "Precipitacao_mm"]].copy()

# Merge por país/ano (inner para manter só interseção)
df_unificado = base_rice.merge(
    base_clima,
    left_on=["Pais_norm", "Year"],
    right_on=["Pais_norm", "Ano"],
    how="inner"
)

# Selecionar e renomear colunas finais
df_unificado = df_unificado[[
    "Area", "Year", "Production (t)", "Area harvested (ha)", "Yield (kg/ha)",
    "Temp_Media_C", "Precipitacao_mm"
]].rename(columns={"Area": "Pais"})

# Garantir tipos corretos
df_unificado["Year"] = pd.to_numeric(df_unificado["Year"], errors="coerce").astype(int)

# 🔧 Arredondar para 2 casas decimais as colunas numéricas solicitadas
colunas_para_2_casas = [
    "Production (t)", "Area harvested (ha)", "Yield (kg/ha)",
    "Temp_Media_C", "Precipitacao_mm"
]
for col in colunas_para_2_casas:
    if col in df_unificado.columns:
        df_unificado[col] = df_unificado[col].round(2)

# Opcional: exibir sempre com 2 casas
pd.set_option('display.float_format', '{:.2f}'.format)

# Salvar CSV final
out_path = "dataset_rice_clima.csv"
df_unificado.to_csv(out_path, index=False)

print(f"Dataset unificado salvo: {out_path}")
print("Formato final:", df_unificado.shape)
display(df_unificado.head(10))

Dataset unificado salvo: dataset_rice_clima.csv
Formato final: (156, 7)


,Pais,Year,Production (t),Area harvested (ha),Yield (kg/ha),Temp_Media_C,Precipitacao_mm
0,Argentina,2011,1748075.00,257445.00,6790.10,16.85,22.72
1,Bolivia (Plurinational State of),2011,428007.00,178273.00,2400.90,25.68,26.22
2,Brazil,2011,13476994.00,2752891.00,4895.60,25.44,79.29
3,Chile,2011,130375.00,25121.00,5189.90,16.15,6.10
4,Colombia,2011,2009945.00,507709.00,3958.90,25.20,115.21
5,Ecuador,2011,1185900.00,330000.00,3593.60,22.51,98.61
6,Guyana,2011,619198.00,140400.00,4410.20,25.50,68.58
7,Paraguay,2011,408246.00,78606.00,5193.60,24.44,48.79
8,Peru,2011,2624458.00,359612.00,7298.00,9.85,13.06
9,Suriname,2011,235298.00,56930.00,4133.10,24.76,81.36
